# 🤖 Interactive FinOps Analysis with Gemini 2.5 Pro

This notebook provides a truly interactive experience for analyzing your cloud cost data using Gemini 2.5 Pro. Use the widgets below to ask your own questions and leverage advanced features like Code Execution and URL Context.

## ⚠️ Important: Setup and Authentication

Ensure you have run `bash scripts/setup_gcp_notebook.sh` and are authenticated to Google Cloud (`gcloud auth application-default login`).

## 📚 Step 1: Configuration, Imports, and Initialization

In [ ]:
# @markdown Specify the path to your configuration file.CONFIG_PATH = 'config.yaml' # @param {type:"string"}import loggingimport pandas as pdimport ipywidgets as widgetsfrom IPython.display import Markdown, display, clear_outputfrom finops_analysis_platform.config_manager import ConfigManagerfrom finops_analysis_platform.data_loader import load_data_from_configfrom finops_analysis_platform.gemini_service import initialize_gemini, generate_contentfrom finops_analysis_platform.core import CUDAnalyzerfrom google.genai.types import Tool, ToolCodeExecution, UrlContextlogging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')config_manager = ConfigManager(config_path=CONFIG_PATH)project_id = config_manager.get('gcp.project_id')location = config_manager.get('gcp.location', 'us-central1')client = initialize_gemini(project_id=project_id, location=location)print(f"✅ Gemini client initialized for project '{project_id}'.")

## 📂 Step 2: Load Data

In [ ]:
data = load_data_from_config(config_manager)billing_df = data.get('billing')recommendations_df = data.get('recommendations')print("✅ Data loaded successfully. The billing data is available in the `billing_df` DataFrame.")

## 💻 Step 3: Interactive Analysis with Code Execution

Ask a question about your billing data. Gemini will write and execute code to answer it. **Reference the data as `billing_df`.**

In [ ]:
code_prompt_input = widgets.Textarea(    value='Analyze the billing data in `billing_df`. What are the top 3 services by cost? Display the result as a table.',    placeholder='Type your prompt here... e.g., Show me the total cost per project.',    description='Prompt:',    layout={'width': '100%', 'height': '100px'})code_run_button = widgets.Button(description="Run Analysis")code_output_area = widgets.Output()def run_code_execution(b):    with code_output_area:        clear_output()        print("Generating and executing code...")        code_execution_tool = Tool(code_execution=ToolCodeExecution())        response = generate_content(client, 'gemini-2.5-pro', code_prompt_input.value, tools=[code_execution_tool])        clear_output()        if response and response.candidates:            for part in response.candidates[0].content.parts:                if part.executable_code:                    display(Markdown(f"**Generated Code:**\n```python\n{part.executable_code.code}\n```"))                if part.code_execution_result:                    display(Markdown(f"**Execution Result:**\n```\n{part.code_execution_result.output}\n```"))        else:            print("Failed to get a response from the API.")code_run_button.on_click(run_code_execution)display(code_prompt_input, code_run_button, code_output_area)

## 🔗 Step 4: Interactive Analysis with URL Context

Ask a question based on the content of a public URL.

In [ ]:
url_input = widgets.Text(value='https://cloud.google.com/compute/all-pricing', description='URL:')url_prompt_input = widgets.Textarea(    value='Summarize the key differences in pricing between N2, E2, and C2 machine types.',    placeholder='Type your question about the URL content... e.g., What is the main point of this article?',    description='Prompt:',    layout={'width': '100%', 'height': '80px'})url_run_button = widgets.Button(description="Run Analysis")url_output_area = widgets.Output()def run_url_analysis(b):    with url_output_area:        clear_output()        print("Analyzing URL content...")        url_context_tool = Tool(url_context=UrlContext())        prompt = f"Based on this URL: {url_input.value}, {url_prompt_input.value}"        response = generate_content(client, 'gemini-2.5-pro', prompt, tools=[url_context_tool])        clear_output()        if response and response.text:            display(Markdown(response.text))        else:            print("Failed to get a response from the API.")url_run_button.on_click(run_url_analysis)display(url_input, url_prompt_input, url_run_button, url_output_area)

## 💎 Step 5: Advanced AI Portfolio Optimization

Leverage Gemini's analytical capabilities to get a sophisticated, risk-adjusted CUD portfolio recommendation. This uses the new `generate_cud_portfolio_optimization` feature.

In [ ]:
portfolio_button = widgets.Button(description="Generate AI Portfolio Recommendation")portfolio_output_area = widgets.Output()def run_portfolio_optimization(b):    with portfolio_output_area:        clear_output()        print("Running advanced portfolio analysis...")        analyzer = CUDAnalyzer(config_manager=config_manager, billing_data=billing_df)        # We need the savings_by_machine data first        machine_distribution = analyzer._analyze_machine_distribution()        savings_by_machine = analyzer._calculate_savings_by_machine(machine_distribution)        ai_portfolio = analyzer.generate_cud_portfolio_optimization(savings_by_machine)        clear_output()        if ai_portfolio and 'error' not in ai_portfolio:            display(Markdown(f"### AI-Powered Portfolio Strategy\n**Summary:** {ai_portfolio.get('strategy_summary', 'N/A')}"))            display(pd.DataFrame(ai_portfolio.get('portfolio', [])))        else:            display(Markdown(f"**Could not generate AI portfolio:**\n`{ai_portfolio}`"))portfolio_button.on_click(run_portfolio_optimization)display(portfolio_button, portfolio_output_area)